In [88]:
print("testing the kt3d function using some data from Geostats Guy")

testing the kt3d function using some data from Geostats Guy


In [98]:
import geostatspy.GSLIB as GSLIB    
# GSLIB utilies, visualization and wrapper
import geostatspy.geostats as geostats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial as sp  # for fast nearest neighbor search

IndentationError: unindent does not match any outer indentation level (geostats.py, line 364)

In [99]:



# *** all parameters have been copied over form kb2d with respective z parameter added if necessary
# we can probably delete a couple of these if we don't do block kriging
def kt3d (
    df,
    xcol,
    ycol,
    zcol,
    vcol,
    tmin,
    tmax,
    nx,
    xmn,
    xsiz,
    ny,
    ymn,
    ysiz,
    nz,
    zmn,
    zsiz,
    nxdis,
    nydis,
    nzdis,
    ndmin,
    ndmax,
    radius,
    ktype,
    skmean,
    vario,
):
    UNEST = -999.
    EPSLON = 1.0e-10
    # VERSION = 2.907
    # first = True
    PMX = 9999.0    
    MAXSAM = ndmax + 1
    MAXDIS = nxdis * nydis * nzdis
    MAXKD = MAXSAM + 1
    MAXKRG = MAXKD * MAXKD

# Allocate the needed memory:   
    xdb = np.zeros(MAXDIS)
    ydb = np.zeros(MAXDIS)
    zdb = np.zeros(MAXDIS)
    xa = np.zeros(MAXSAM)
    ya = np.zeros(MAXSAM)
    za = np.zeros(MAXSAM)
    vra = np.zeros(MAXSAM)
    dist = np.zeros(MAXSAM)
    nums = np.zeros(MAXSAM)
    r = np.zeros(MAXKD)
    rr = np.zeros(MAXKD)
    s = np.zeros(MAXKD)
    a = np.zeros(MAXKRG)
    kmap = np.zeros((nx,ny,nz))
    vmap = np.zeros((nx,ny,nz))

    # trim values outside tmin and tmax
    df_extract = df.loc[(df[vcol] >= tmin) & (df[vcol] <= tmax)]  
    assert df[vcol].max() <= tmax and df[vcol].min() >= tmin
    nd = len(df_extract)
    ndmax = min(ndmax,nd)
    x = df_extract[xcol].values
    y = df_extract[ycol].values
    z = df_extract[zcol].values
    vr = df_extract[vcol].values


    # Allocate the needed memory:   
    xdb = np.zeros(MAXDIS)
    ydb = np.zeros(MAXDIS)
    zdb = np.zeros(MAXDIS)
    xa = np.zeros(MAXSAM)
    ya = np.zeros(MAXSAM)
    za = np.zeros(MAXSAM)
    vra = np.zeros(MAXSAM)
    dist = np.zeros(MAXSAM)
    nums = np.zeros(MAXSAM)
    r = np.zeros(MAXKD)
    rr = np.zeros(MAXKD)
    s = np.zeros(MAXKD)
    a = np.zeros(MAXKRG)
    kmap = np.zeros((nx,ny,nz))
    vmap = np.zeros((nx,ny,nz)) 

# set up tree for nearest neighbor search
    dp = list((z[i], y[i], x[i]) for i in range(0,nd))
    data_locs = np.column_stack((z, y, x))
    tree = sp.cKDTree(data_locs, leafsize=16, compact_nodes=True, copy_data=False, balanced_tree=True) #why this error?

# Summary statistics for the data after trimming
    avg = vr.mean()
    stdev = vr.std()
    ss = stdev**2.0 # variance
    vrmin = vr.min()
    vrmax = vr.max()

# load the variogram
    nst = vario['nst'] # num structures
    cc = np.zeros(nst) # variance contribution
    aa = np.zeros(nst) # major range
    it = np.zeros(nst) # type vario structure
    aa = np.zeros(nst) # major range
    ang_azi = np.zeros(nst) # azimuth
    ang_dip = np.zeros(nst) # dip
    anis = np.zeros(nst) # anistropy ratio w/ minor range
    anis_v = np.zeros(nst) # anistropy ratio w/ vertical range
    
    # only works w/ nst == 1 or nst == 2
    print("vario")
    print(vario)
    c0 = vario['nug'] 
    it[0] = vario['it1'] 
    cc[0] = vario['cc1'] 
    ang_azi[0] = vario['azi1']
    ang_dip[0] = vario['dip1']
    aa[0] = vario['hmax']
    anis[0] = vario['hmed1']/vario['hmax1']
    anis_v[0] = vario['hmin1']/vario['hmax1']
    if nst == 2:
        cc[1] = vario['cc2']
        it[1] = vario['it2']
        ang_azi[1] = vario['azi2']
        ang_dip[1] = vario['dip1']
        aa[1] = vario['hmax']
        anis[1] = vario['hmed2']/vario['hmax2']
        anis_v[1] = vario['hmin2']/vario['hmax2']

    #rotmat, maxcov = setup_rotmat_3D(c0, nst, it, cc, ang_azi, ang_dip, PMX)
    #cbb = maxcov

    # TODO Set up discretization points per block (bock kriging)

    # need to set up numPy cube grid
    cubeGrid = np.ndarray(shape=(nx,ny,nz), dtype=float, order='C') 
    print(cubeGrid)
    # main loop over all points:
    nk = 0
    ak = 0.0
    vk = 0.0
    for iz in range(0,nz):
        zloc = zmn + (iz-0)*zsiz 
        for iy in range(0,ny):
            yloc = ymn + (iy-0)*ysiz
            for ix in range (0,nz):     #triple nested for loop that loops through points and adds to matrices
                xloc = xmn + (ix-0)*xsiz
                current_node = (zloc, yloc,xloc) # xloc, yloc, zloc centroid of current cell


    # Find the nearest samples within each octant: First initialize
# the counter arrays:
                na = -1   # accounting for 0 as first index
                dist.fill(1.0e+20)
                nums.fill(-1)
                dist, nums = tree.query(current_node,ndmax) # use kd tree for fast nearest data search
                # remove any data outside search radius
                na = len(dist)
                nums = nums[dist<radius]
                dist = dist[dist<radius] 
                na = len(dist)        

                # Is there enough samples?
                if na + 1 < ndmin:   # accounting for min index of 0
                    est  = UNEST
                    estv = UNEST
                    print('UNEST at ' + str(ix) + ',' + str(iy) + ','+str(iz)) 
                else:

                    # Put coordinates and values of neighborhood samples into xa,ya,za,vra:
                    for ia in range(0,na):
                        jj = int(nums[ia])
                        xa[ia]  = x[jj] #why this error?
                        ya[ia]  = y[jj]
                        za[ia]  = z[jj]
                        vra[ia] = vr[jj] #what is vra??
                    
                    # Handle the situation of only one sample:
                    if na == 0:  # accounting for min index of 0 - one sample case na = 0
                        cb1 = geostats.cova3(xa[0],ya[0],za[0],xa[0],ya[0],za[0],nst,c0,PMX,cc,aa,it,ang,anis,rotmat,maxcov)

                        xx  = xa[0] - xloc
                        yy  = ya[0] - yloc
                        zz  = za[0] - zloc

                # Is there enough samples?
                if na + 1 < ndmin:   # accounting for min index of 0
                    est  = UNEST
                    estv = UNEST
                    print('UNEST at ' + str(ix) + ',' + str(iy) + ','+str(iz)) 
                else:
                    # Put coordinates and values of neighborhood samples into xa,ya,za,vra:
                    for ia in range(0,na):
                        jj = int(nums[ia])
                        xa[ia]  = x[jj] #why this error?
                        ya[ia]  = y[jj]
                        za[ia]  = z[jj]
                        vra[ia] = vr[jj] #what is vra??
                    
                    # Handle the situation of only one sample:
                    if na == 0:  # accounting for min index of 0 - one sample case na = 0
                        cb1 = geostats.cova3(xa[0],ya[0],za[0],xa[0],ya[0],za[0],nst,c0,PMX,cc,aa,it,ang,anis,rotmat,maxcov)

                        xx  = xa[0] - xloc
                        yy  = ya[0] - yloc
                        zz  = za[0] - zloc

                    # Establish Right Hand Side Covariance:
                        if ndb <= 1:
                            cb = geostats.cova3(xx,yy,zz,xdb[0],ydb[0],zdb[i],nst,c0,PMX,cc,aa,it,ang,anis,rotmat,maxcov)
                        else:    
                            cb  = 0.0
                            for i in range(0,ndb):                  
                                cb = cb + geostats.cova3(xx,yy,zz,xdb[i],ydb[i],zdb[i],nst,c0,PMX,cc,aa,it,ang,anis,rotmat,maxcov)
                                dx = xx - xdb(i)
                                dy = yy - ydb(i)
                                dz = zz - zdb(i)
                                if (dx*dx+dy*dy+dz*dz) < EPSLON:
                                    cb = cb - c0
                                cb = cb / real(ndb)
                        if ktype == 0:
                            s[0] = cb/cbb
                            est  = s[0]*vra[0] + (1.0-s[0])*skmean
                            estv = cbb - s[0] * cb
                        else:
                            est  = vra[0]
                            estv = cbb - 2.0*cb + cb1
                    else:
                    # Solve the Kriging System with more than one sample:
                        neq = na + ktype # accounting for first index of 0
                        # print('NEQ' + str(neq))
                        nn  = (neq + 1)*neq/2   

                        # Set up kriging matrices:
                        a = np.zeroes((na,na))
                        r = np.zeroes(na)

                        # Establish Left Hand Side Covariance Matrix:

                        for j in range(0,na):
                            for i in range(0,na): #loops through and adds covariances into matrix
                                a[j][i] = geostats.cova3(xa[i],ya[i],za[i],xa[j],ya[j],za[j],nst,c0,PMX,cc,aa,it,ang_azi,anis,rotmat,maxcov) 
                            r[j] = geostats.cova3(xloc,yloc,zloc,xa[j],ya[j],za[j],nst,c0,PMX,cc,aa,it,ang_azi,anis,rotmat,maxcov) 

# Write a warning if the matrix is singular:
                        if ising != 0:
                            print('WARNING KT3D: singular matrix')
                            print('              for block' + str(ix) + ',' + str(iy) + ',' + str(iz))
                            est  = UNEST
                            estv = UNEST
                        else:

# Compute the estimate and the kriging variance:
                            est  = 0.0
                            estv = cbb
                            sumw = 0.0
                            if ktype == 1: 
                                estv = estv - (s[na])*unbias
                            for i in range(0,na):                          
                                sumw = sumw + s[i]
                                est  = est  + s[i]*vra[i]
                                estv = estv - s[i]*rr[i]
                            if ktype == 0: 
                                est = est + (1.0-sumw)*skmean
                kmap[nz-iz-1, ny-iy-1,ix] = est
                vmap[nz-iz-1, ny-iy-1,ix] = estv
                if est > UNEST:
                    nk = nk + 1
                    ak = ak + est
                    vk = vk + est*est
# TODO Set up kriging matrices:

    return kmap, vmap   


In [100]:
skmean_por = 0.10; skmean_perm = 65.0      # simple kriging mean (used if simple kriging is selected below)
ktype = 1                                  # kriging type, 0 - simple, 1 - ordinary
radius = 100                               # search radius for neighbouring data
nxdis = 1; nydis = 1 ; nzdis = 1         # number of grid discretizations for block kriging (not tested)
ndmin = 0; ndmax = 10                      # minimum and maximum data for an estimate
tmin = 0.0                                 # minimum property value
xsiz = 10; ysiz = 10 ; zsiz = 10                   # cell size
nx = 100; ny = 100; nz = 100                      # number of cells
xmn = 5; ymn = 5 ; zmn = 5                       # grid origin, location center of lower left cell
por_vario = GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=100,hmin1=100) # porosity variogram
por_vario["dip1"] = 0
por_vario["hmax"] = 0
por_vario["hmed1"] = 0
por_vario["hmax1"] = 1


In [101]:
df = pd.read_csv("completions_production.csv")
por_kmap, por_vmap = kt3d ( df, 'X', 'Y', 'Z', 'Porosity',  tmin, tmax, nx,
    xmn,  xsiz,  ny,  ymn,   ysiz,  nz,  zmn,  zsiz,  nxdis, nydis,  nzdis, ndmin, ndmax, radius, ktype, skmean_por, por_vario)
print(por_kmap)
print(por_vmap)



vario
{'nug': 0.0, 'nst': 1, 'it1': 1, 'cc1': 1.0, 'azi1': 0, 'hmaj1': 100, 'hmin1': 100, 'it2': 1, 'cc2': 0, 'azi2': 0, 'hmaj2': 0, 'hmin2': 0, 'dip1': 0, 'hmax': 0, 'hmed1': 0, 'hmax1': 1}
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0.

NameError: name 'geostats' is not defined